In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import json
import gradio as gr

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [6]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [13]:
seats_available = {"london": 25, "paris": 30, "tokyo": 10, "berlin": 0}

def get_seats_available(destination_city):
    print(f"Tool get_seats_available called for {destination_city}")
    city = destination_city.lower()
    return seats_available.get(city, "Unknown")

In [8]:
# There's a particular dictionary structure that's required to describe our function:

seats_function = {
    "name": "get_seats_available",
    "description": "Get the seats of a return ticket to the destination city. Call this whenever you need to know the ticket available, for example when a customer asks 'I want to make a reservation'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [9]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function},
         {"type": "function", "function": seats_function}]

In [10]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")

    # Decide which function to call based on the tool name
    function_name = tool_call.function.name

    if function_name == "get_ticket_price":
        result = get_ticket_price(city)
        response_content = {
            "destination_city": city,
            "price": result
        }

    elif function_name == "get_seats_available":
        result = get_seats_available(city)
        response_content = {
            "destination_city": city,
            "seats_available": result
        }

    else:
        # Unknown tool function
        result = "Unknown tool"
        response_content = {
            "destination_city": city,
            "error": result
        }

    response = {
        "role": "tool",
        "content": json.dumps(response_content),
        "tool_call_id": tool_call.id
    }

    return response, city


In [11]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [14]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Berlin
Tool get_seats_available called for Berlin
Tool get_ticket_price called for London
Tool get_seats_available called for London
Tool get_ticket_price called for Berlin
Tool get_seats_available called for Berlin
Tool get_ticket_price called for London
Tool get_seats_available called for London
Tool get_seats_available called for London
Tool get_ticket_price called for Berlin


Traceback (most recent call last):
  File "c:\Users\iara.santos\AppData\Local\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\iara.santos\AppData\Local\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\iara.santos\AppData\Local\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\iara.santos\AppData\Local\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1700, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\iara.santos\AppData\Local\anaconda3\envs\llms\Lib\site-packages